In [14]:
import pandas as pd
import os
from pathlib import Path

In [15]:
INPUT_PATH_1 = Path('/mnt/synthetic_data/lung/data/processed/3_postprocess')
INPUT_PATH_2 = Path('/mnt/synthetic_data/lung/data/processed/3_postprocess/after_QC')

In [16]:
path2_tables = []
for i in range(len(os.listdir(INPUT_PATH_2))):
    path2_tables.append(('_').join(os.listdir(INPUT_PATH_2)[i].split('_')[:3]))
path2_tables = list(set(path2_tables))

In [17]:
path1_tables = []
for i in range(len(os.listdir(INPUT_PATH_1))-1):
    path1_tables.append(('_').join(os.listdir(INPUT_PATH_1)[i].split('_')[:3]))
path1_tables = list(set(path1_tables))

In [18]:
def exclude_pt_no(standard, target):
    st_no = standard['PT_SBST_NO'].unique()
    tr_no = target['PT_SBST_NO'].unique()
    diff = (set(tr_no).difference(set(st_no)))
    
    return target[~target['PT_SBST_NO'].isin(diff)]    

In [19]:
path2_tables

['LUNG_PT_BSNF',
 'LUNG_TRTM_CASB',
 'LUNG_TRTM_RD',
 'LUNG_PTH_BPSY',
 'LUNG_PTH_SRGC']

In [21]:
OUTPUT_PATH = Path('/mnt/synthetic_data/lung/data/processed/3_postprocess/final_lung_synthetic_data')
for epsilon in [0.1,1,10,100,1000]:
    bsnf = pd.read_excel(INPUT_PATH_2.joinpath('LUNG_PT_BSNF'+f'_{epsilon}.xlsx'))
    for table in path2_tables[1:]:
        try:
            tr = pd.read_excel(INPUT_PATH_2.joinpath(table+f'_{epsilon}.xlsx'))
            file_type = 'xlsx'
        except:
            tr = pd.read_csv(INPUT_PATH_2.joinpath(table+f'_{epsilon}.csv'))
            file_type = 'csv'
        
        tr = tr.drop(tr.filter(like='Un'),axis=1)
        tr = exclude_pt_no(bsnf, tr)
        
        if file_type == 'xlsx':
            tr.to_excel(OUTPUT_PATH.joinpath(f'{epsilon}/{table}_{epsilon}.xlsx'),index=False)
        elif file_type == 'csv':
            tr.to_csv(OUTPUT_PATH.joinpath(f'{epsilon}/{table}_{epsilon}.csv'),index=False)
        else:
            print('File Type Error')       
        

In [23]:
table_1 = list(set(path1_tables).difference(set(path2_tables)))
table_1

['LUNG_EX_PLMN',
 'final_lung_synthetic',
 'LUNG_PTH_MLCR',
 'after_QC',
 'LUNG_OPRT_NFRM',
 'LUNG_PTH_MNTY',
 'LUNG_DEAD_NFRM']

In [24]:
table_1 = list(set(path1_tables).difference(set(path2_tables)))
del table_1[1]
del table_1[2]

for epsilon in [0.1,1,10,100,1000]:
    bsnf = pd.read_excel(INPUT_PATH_2.joinpath('LUNG_PT_BSNF'+f'_{epsilon}.xlsx'))
    for table in table_1:
        try:
            tr = pd.read_excel(INPUT_PATH_1.joinpath(table+f'_{epsilon}.xlsx'))
            file_type = 'xlsx'
        except:
            tr = pd.read_csv(INPUT_PATH_1.joinpath(table+f'_{epsilon}.csv'))
            file_type = 'csv'
        
        tr = tr.drop(tr.filter(like='Un'),axis=1)
        tr = exclude_pt_no(bsnf, tr)
        
        if file_type == 'xlsx':
            tr.to_excel(OUTPUT_PATH.joinpath(f'{epsilon}/{table}_{epsilon}.xlsx'),index=False)
        elif file_type == 'csv':
            tr.to_csv(OUTPUT_PATH.joinpath(f'{epsilon}/{table}_{epsilon}.csv'),index=False)
        else:
            print('File Type Error')       
        

In [25]:
INPUT_PATH_3 = Path('/mnt/synthetic_data/lung/data/processed/3_postprocess/final_lung_synthetic_data')
tables = ['LUNG_PTH_BPSY',
 'LUNG_PTH_MNTY',
 'LUNG_OPRT_NFRM',
 'LUNG_TRTM_RD',
 'LUNG_EX_PLMN',
 'LUNG_PTH_SRGC',
 'LUNG_DEAD_NFRM',
 'LUNG_PTH_MLCR']

In [27]:
for epsilon in [0.1,1,10,100,1000]:
    bsnf = pd.read_excel(INPUT_PATH_2.joinpath('LUNG_PT_BSNF'+f'_{epsilon}.xlsx'))
    bsnf = bsnf.drop(bsnf.filter(like='Un'),axis=1)
    merged_ymd = pd.DataFrame(bsnf['PT_SBST_NO'])
    for table in tables:
        try:
            temp_pd = pd.read_excel(INPUT_PATH_3.joinpath(f'{epsilon}/{table}_{epsilon}.xlsx'))
        except:
            temp_pd = pd.read_csv(INPUT_PATH_3.joinpath(f'{epsilon}/{table}_{epsilon}.csv'))
        sel_col = list(temp_pd.filter(like='YMD').columns)
        sel_col.append('PT_SBST_NO')
        ymd_pd = temp_pd[sel_col]
        ymd_pd = ymd_pd.drop_duplicates(subset='PT_SBST_NO',keep='last')
        merged_ymd = pd.merge(merged_ymd,ymd_pd,how='left',on='PT_SBST_NO')
    bsnf['CENTER_LAST_VST_YMD'] = merged_ymd.drop('PT_SBST_NO',axis=1).astype('datetime64[ns]').max(axis = 1)
    bsnf['OVRL_SRVL_DTRN_DCNT'] = (bsnf['CENTER_LAST_VST_YMD'] - bsnf['BSPT_FRST_DIAG_YMD']).dt.days
    print((bsnf['PT_SBST_NO']).nunique())
    bsnf.to_excel(OUTPUT_PATH.joinpath(f'{epsilon}/LUNG_PT_BSNF_{epsilon}.xlsx'),index=False)

39690
39580
36170
31215
30280


In [28]:
bsnf = pd.read_excel(INPUT_PATH_2.joinpath('LUNG_PT_BSNF'+f'_1000.xlsx'))

In [29]:
bsnf

,Unnamed: 0.1,Unnamed: 0,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,BSPT_SEX_CD,BSPT_BRYM,BSPT_FRST_DIAG_YMD,BSPT_FRST_DIAG_CD,BSPT_FRST_DIAG_NM,...,BSPT_FRST_RDT_STRT_YMD,BSPT_DEAD_YMD,CENTER_LAST_VST_YMD,OVRL_SRVL_DTRN_DCNT,BSPT_STAG_CLSF_CD,BSPT_STAG_VL,BSPT_T_STAG_VL,BSPT_N_STAG_VL,BSPT_M_STAG_VL,CRTN_DT
0,0,0,0,2-2222-02-22,0,F,2179-03-04,2222-02-22,C343,"Lower lobe, bronchus or lung",...,NaT,2231-10-17,NaN,NaN,NaN,2,2,0,0,2023-10-20
1,3,3,0,2-2222-02-22,100,M,2167-03-07,2222-02-22,C341,"Upper lobe, bronchus or lung",...,NaT,NaT,NaN,NaN,NaN,1,1,0,0,2023-10-20
2,6,6,0,2-2222-02-22,10000,F,2142-03-13,2222-02-22,C343,"Lower lobe, bronchus or lung",...,NaT,NaT,NaN,NaN,NaN,4,4,3,1,2023-10-20
3,11,11,0,2-2222-02-22,10002,F,2178-03-04,2222-02-22,C343,"Lower lobe, bronchus or lung",...,NaT,NaT,NaN,NaN,NaN,2,1,1,0,2023-10-20
4,13,13,0,2-2222-02-22,10004,F,2171-03-06,2222-02-22,C343,"Lower lobe, bronchus or lung",...,NaT,NaT,NaN,NaN,NaN,3,3,2,0,2023-10-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30275,60344,60344,0,2-2222-02-22,9992,M,2169-03-06,2222-02-22,C342,"Middle lobe, bronchus or lung",...,NaT,NaT,NaN,NaN,NaN,2,2,1,0,2023-10-20
30276,60347,60347,0,2-2222-02-22,9994,M,2147-03-12,2222-02-22,C341,"Upper lobe, bronchus or lung",...,2222-02-22,2226-10-10,NaN,NaN,NaN,2,2,1,0,2023-10-20
30277,60350,60350,0,2-2222-02-22,9995,F,2171-03-06,2222-02-22,C341,"Upper lobe, bronchus or lung",...,NaT,NaT,NaN,NaN,NaN,3,2,2,0,2023-10-20
30278,60351,60351,0,2-2222-02-22,9996,M,2158-03-09,2222-02-22,C343,"Lower lobe, bronchus or lung",...,NaT,2224-03-05,NaN,NaN,NaN,3,x,3,0,2023-10-20


In [30]:
a = [39690,39580,36170,31215,30280]
sum(a)

176935